<H3>PRI 2023/24: first project delivery</H3>

**GROUP 11**
- Francisco Martins, 99068
- Tunahan Güneş, 108108
- Sebastian Weidinger, 111612

<H3>Part I: demo of facilities</H3>

In [1]:
import os

In [2]:
def read_files(path):
    texts = []
    file_paths = []
    for folder in os.listdir(path):
        category_path = os.path.join(path, folder)
        texts.append([])
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            file_paths.append(file_path)
            with open(file_path, "r", errors="ignore") as f:
                text = f.read()
                texts[-1].append(text)
                
    print("Number of Categories:",len(os.listdir(path)))
    for i in range(len(os.listdir(path))):
        print("Number of Articles in", "'"+os.listdir(path)[i]+"'", "Category:",len(texts[i]))
    return file_paths, texts

In [3]:
dataset_path = os.path.join("BBC News Summary", "BBC News Summary", "News Articles")
print("Dataset path:", dataset_path)
file_paths, categorized_articles = read_files(dataset_path)

Dataset path: BBC News Summary/BBC News Summary/News Articles


Number of Categories: 5
Number of Articles in 'tech' Category: 401
Number of Articles in 'entertainment' Category: 386
Number of Articles in 'sport' Category: 511
Number of Articles in 'business' Category: 510
Number of Articles in 'politics' Category: 417


In [4]:
#Examplary text. The structure of the read file is: articles[category_no][document_no]. 
print(categorized_articles[0][0])
print(file_paths[508:512])

Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye following an injury, wh

A) **Indexing** (preprocessing and indexing options)

In [5]:
#code, statistics and/or charts here

imports

In [6]:
import time 
from typing import Union
import nltk
import numpy as np
import math
import torch
import sklearn
from nltk.tokenize import RegexpTokenizer
from collections import Counter, defaultdict
from tabulate import tabulate
from transformers import BertTokenizer, BertModel

/home/mono/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# flatten list to get uncategorized collection 
def flatten(lists) -> list: 
    return [element for sublist in lists for element in sublist]

articles = flatten(categorized_articles)
N = len(articles)
dict_path_to_articleID = {path:i for i, path in enumerate(file_paths)}
def map_path_to_articleID(path):
    path = os.path.normpath(path)
    return dict_path_to_articleID.get(path)

In [8]:
N

2225

# Inverted Index Structure 
 
Each term points to a dictionary of document identifier and the term frequency in the document.

t1 -> {doc1: TF, doc5: TF, ...}\
t2 -> {doc7: TF, doc8: TF, ...}\
...
t2 -> [DF, {doc7: [TF_(t2, doc7), {s1: TF, s4: TF, ...}], doc8: [TF_(t2, doc8), {s2: TF, s4: TF, ...}], ...}]\

use class structure

TODO: 
* Optimize structure?
    * Is there a more efficient way? 
    * Add maybe pointers to sentences and their term frequency? -> Faster?

In [9]:
max_width = 20

In [10]:
class TermFrequencies: 
    def __init__(self) -> None:
        self.tf_d_t = 0
        self.sent_tf = list()

    def add_sentence(self, sent_number, term_frequency):
        self.sent_tf.append((sent_number, term_frequency))
    
    def __repr__(self):
        padding = 5 - len(str(self.tf_d_t))
        return f'TF_d_t: {self.tf_d_t}{" " * padding}TF_per_sentence: {self.sent_tf}'

In [11]:
class InvertedIndexEntry:
    def __init__(self) -> None:
        self.df_term = 0
        self.term_dict = defaultdict(TermFrequencies)
    
    def get_document(self, document):
        return self.term_dict.get(document, None)

    def get_or_default_document(self, document):
        return self.term_dict[document]

    def update_document(self, document, new_value):
        self.term_dict[document] = new_value
    
    def __repr__(self):
        out = f'Document Frequency: {self.df_term}\n {" " * (max_width+2)} Term frequencies:\n'
        for doc_number, tfs in self.term_dict.items():
            padding = 5 - len(str(doc_number))
            out += f'{" " * (max_width + 3)} Doc {doc_number}{" " * padding}→ {tfs}\n'
        return out
    
    def calculate_df(self):
        self.df_term = len(self.term_dict)

In [12]:
class InvertedIndex:
    def __init__(self, collection_size) -> None:
        self.inverted_index = defaultdict(InvertedIndexEntry)
        self.sentence_lengths = list()
        self.indexing_time = 0
        self.N = collection_size
    
    def __repr__(self):
        out = f'Time to index: {self.indexing_time}\nInverted Index:\n'
        for term, entry in self.inverted_index.items():
            padding = max_width - len(term)
            out += f'{term} {" " * padding} → {entry}\n'
        return out

    def get_or_default(self, term, document):
        return self.inverted_index[term].get_or_default_document(document)
    
    def update(self, term, document, new_value):
        self.inverted_index[term].update_document(document, new_value)
    
    def set_indexing_time(self, indexing_time):
        self.indexing_time = indexing_time
    
    def calculate_dfs(self):
        for entry in self.inverted_index.values():
            entry.calculate_df()  
    
    def get_sentence_lengths(self, document):
        return self.sentence_lengths[document]

    def get_document_info(self, document):          
        info = {'Vocabulary': [], 'DF_t': [], 'TF_d_t': [], 'TF/sentence': []}
        for term, entry in self.inverted_index.items():
            doc_tfs = entry.get_document(document)
            if doc_tfs == None:
                continue
            info['Vocabulary'].append(term)
            info['DF_t'].append(entry.df_term)
            info['TF_d_t'].append(doc_tfs.tf_d_t)
            info['TF/sentence'].append(doc_tfs.sent_tf)
        return info
    
    def doc_to_string(self, document):
        out = f'Document {document} vocabulary and term frequencies:\n'
        info = self.get_document_info(document)
        table = zip(*info.values())
        headers = info.keys()
        return out + tabulate(table, headers, tablefmt="pretty")


In [13]:
'''
indexing(D,args)
    @input document collection D and optional arguments on text preprocessing

    @behavior preprocesses the collection and, using existing libraries, 
    builds an inverted index with the relevant statistics for the subsequent summarization functions
    
    @output pair with the inverted index I and indexing time
'''
def indexing(articles, args=None) -> InvertedIndex:
    start_time = time.time()
    inverted_index = InvertedIndex(len(articles))

    # tokenizer split words and keep hyphons e.g. state-of-the-art
    tokenizer = RegexpTokenizer(r'[\w|-]+')

    # loop through collection 
    for article_id, article in enumerate(articles): 
        # split into sentences
        sents = nltk.sent_tokenize(article)
        # remove title (not needed for summarization task)
        sents = sents[1:]
        # save words per sent in list 
        tokenized_sentences = [tokenizer.tokenize(sent.lower()) for sent in sents]
        # calculate length of the sentences in the article
        sent_lengths = [len(sentence_terms) for sentence_terms in tokenized_sentences]
        inverted_index.sentence_lengths.append(sent_lengths)
        # count the term frequencies per sentence
        term_counter_per_sent = [Counter(sentence_terms) for sentence_terms in tokenized_sentences]
        for sent_number, term_counter in enumerate(term_counter_per_sent):
            for term in term_counter: 
                tf = term_counter[term]
                term_document_tfs = inverted_index.get_or_default(term, article_id)
                term_document_tfs.tf_d_t += tf 
                term_document_tfs.add_sentence(sent_number, tf)
                inverted_index.update(term, article_id, term_document_tfs)
    inverted_index.calculate_dfs()
    end_time = time.time()
    indexing_time = end_time - start_time
    inverted_index.set_indexing_time(indexing_time)
    return inverted_index


In [14]:
s0 = 'Title. The little white little rabbit. The person played with the ball.'
s1 = 'Title. The white rabbit\'s ball. Rabbit rabbit ball rabbit.'
s2 = 'Title.  White, the little white rabbit. Little, little.'
test = [s0, s1, s2]
I_test = indexing(test)

In [15]:
print(I_test)

Time to index: 0.0060422420501708984
Inverted Index:
the                   → Document Frequency: 3
                        Term frequencies:
                        Doc 0    → TF_d_t: 3    TF_per_sentence: [(0, 1), (1, 2)]
                        Doc 1    → TF_d_t: 1    TF_per_sentence: [(0, 1)]
                        Doc 2    → TF_d_t: 1    TF_per_sentence: [(0, 1)]

little                → Document Frequency: 2
                        Term frequencies:
                        Doc 0    → TF_d_t: 2    TF_per_sentence: [(0, 2)]
                        Doc 2    → TF_d_t: 3    TF_per_sentence: [(0, 1), (1, 2)]

white                 → Document Frequency: 3
                        Term frequencies:
                        Doc 0    → TF_d_t: 1    TF_per_sentence: [(0, 1)]
                        Doc 1    → TF_d_t: 1    TF_per_sentence: [(0, 1)]
                        Doc 2    → TF_d_t: 2    TF_per_sentence: [(0, 2)]

rabbit                → Document Frequency: 3
                        Te

In [16]:
print(I_test.doc_to_string(2))

Document 2 vocabulary and term frequencies:
+------------+------+--------+------------------+
| Vocabulary | DF_t | TF_d_t |   TF/sentence    |
+------------+------+--------+------------------+
|    the     |  3   |   1    |     [(0, 1)]     |
|   little   |  2   |   3    | [(0, 1), (1, 2)] |
|   white    |  3   |   2    |     [(0, 2)]     |
|   rabbit   |  3   |   1    |     [(0, 1)]     |
+------------+------+--------+------------------+


In [17]:
I = indexing(articles)

In [18]:
print(I.sentence_lengths[0:2])

[[32, 14, 27, 20, 40, 23, 29, 30, 18, 12, 31, 25, 16, 24, 24, 14, 23, 41, 37, 41, 31, 25, 21, 7, 17, 40, 6], [18, 18, 17, 12, 14, 10, 15, 7, 15, 32, 18, 15, 24, 18]]


In [19]:
document_path = 'BBC News Summary/BBC News Summary/News Articles\\business\\509.txt'
print(I.doc_to_string(map_path_to_articleID(document_path)))

Document None vocabulary and term frequencies:
+------------+------+--------+-------------+
| Vocabulary | DF_t | TF_d_t | TF/sentence |
+------------+------+--------+-------------+
+------------+------+--------+-------------+


# Summarization 

TF: 
* Document: Term frequencies are assessed on document level.
* Sentence: Term frequencies are assessed on sentence level.

IDF: Inverted document frequencies is assessed on collection level.\
\
Additional parameter "N" and "article_id". Is this allowed?

TODO: 
* Evaluate choice and give reason: 
    * IDF on document level?
    * TF on document level for sentences? 
* "order" parameter o
* BM25
* BERT embedding

In [20]:
def tf_idf_term(N, df_t, tf_t_d):
    return (1 + math.log10(tf_t_d)) * math.log10(N/df_t)

In [52]:
def sort_by_value(d: dict, max_sent: int, reverse=False) -> dict: 
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=reverse)[:max_sent])

In [66]:
def get_embedding(sentence: str, model, tokenizer, max_length=512) -> torch.tensor: 
    encoded_input = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=max_length)
    output = model(**encoded_input)
    embedding = output["pooler_output"].squeeze()
    # mean pooled embedding might be better
    # mean_pooled_embedding = last_hidden_states.mean(axis=1)
    return embedding

In [76]:
'''
summarization(d,p,l,o,I,args)
    @input document d (the index in I/D), maximum number of sentences (p) and/or characters (l), order
    of presentation o (appearance in text vs relevance), inverted index I or the
    collection D, and optional arguments on IR models

    @behavior preprocesses d, assesses the relevance of each sentence in d against I ac-
    cording to args, and presents them in accordance with p, l and o
    
    @output summary s of document d, i.e. ordered pairs (sentence position in d, score)
'''
def summarization(d: int, p: int, l: int, o: int, I_or_D: Union[InvertedIndex, list], **args) -> list:

    if args['model'] != 'BERT':

        ## if we receive the collection instead of the inverted index we must compute it first
        if type(I_or_D) == list:
            I = indexing(I_or_D)         
        else: 
            I = I_or_D
        
        doc_info = I.get_document_info(d)
        sentence_lengths = I.get_sentence_lengths(d)
        term_doc_info = zip(*doc_info.values())    

    scores = defaultdict(int)

    if args['model'] == 'TF-IDF':
        for term, df_t, tf_t_d, tf_per_sentence in term_doc_info:
            rel_t_d = tf_idf_term(I.N, tf_t_d, df_t)
            for sent_number, tf_s_t in tf_per_sentence:
                scores[sent_number] += rel_t_d * (1 + math.log10(tf_s_t))
        # normalization
        for sent_number, score in scores.items():
            scores[sent_number] = score / sentence_lengths[sent_number] 
    
    elif args['model'] == 'BM25':
        pass
    
    elif args['model'] == 'BERT':
        document = I_or_D[d]
        tokenizer = args['bert_tokenizer']
        bert_model = args['bert_model']
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        bert_model.to(device)
        scores = defaultdict(float)
        # sentences 
        sentences = nltk.sent_tokenize(document)
        sentences = sentences[1:]
        num_sentences = len(sentences)
        sent_embeddings = list()
        # every sentences on its own, no padding needed, faster on cpu
        # for gpu batches are better 
        for sent in sentences: 
            sent_embedding = get_embedding(sent, bert_model, tokenizer)
            sent_embeddings.append(sent_embedding)
        # document
        doc_embedding = get_embedding(document, bert_model, tokenizer, max_length=256)
        for sent_id in range(0, num_sentences): 
            sent_vec = sent_embeddings[sent_id]
            score = torch.nn.functional.cosine_similarity(doc_embedding, sent_vec, dim=0)
            scores[sent_id] = score.item()
            
    else:
        raise ValueError("Currently we only support the following models:\n→ TF-IDF\n→ BM-25\n→ BERT")
    
    sorted_scores = sort_by_value(scores, max_sent=p, reverse=True)
    
    return sorted_scores

In [44]:
article_id = 0
print("ORIGINAL DOCUMENT")
print(articles[article_id])
scores = summarization(d=0, p=5, l=1000, o="rel", I_or_D=I, model='TF-IDF')

print("SUMMARY")
sentences = nltk.sent_tokenize(articles[article_id])
sentences = sentences[1:]
for sent_id, score in scores.items(): 
    print(f"{score:.2f}: {sentences[sent_id]}")

ORIGINAL DOCUMENT
Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye follo

In [68]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [77]:
article_id = 0
print("ORIGINAL DOCUMENT")
print(articles[article_id])
scores = summarization(d=0, p=5, l=1000, o="rel", I_or_D=articles, model='BERT', bert_model=bert_model, bert_tokenizer=bert_tokenizer)

print("SUMMARY")
sentences = nltk.sent_tokenize(articles[article_id])
sentences = sentences[1:]
for sent_id, score in scores.items(): 
    print(f"{score:.2f}: {sentences[sent_id]}")

ORIGINAL DOCUMENT
Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye follo

# Keyword Extraction

Calculates the keywords based on the tf-idf of the document.\
\
Additional parameter "N" and "article_id". Is this allowed?

Parameter for including only noun phrases. 

TODO:
* Nouns: just unigrams or also bigrams?
* BM25
* BERT embedding


In [ ]:
from nltk.classify import Senna
from nltk.tag import SennaChunkTagger

In [ ]:
'''
keyword extraction(d,p,I,args)
    @input document d, maximum number of keywords p, inverted index I, and op-
    tional arguments on IR model choices

    @behavior extracts the top informative p keywords in d against I according to args
    
    @output ordered set of p keywords
'''

'\nkeyword extraction(d,p,I,args)\n    @input document d, maximum number of keywords p, inverted index I, and op-\n    tional arguments on IR model choices\n\n    @behavior extracts the top informative p keywords in d against I according to args\n    \n    @output ordered set of p keywords\n'

In [ ]:
def keyword_extraction(article: str, max_num: int, inverted_index: dict, N: int, article_id: int, use_only_nouns=False, args=None) -> dict: 
    # tokenizer split words and keep hyphons e.g. state-of-the-art
    tokenizer = RegexpTokenizer(r'[\w|-]+')
    # tokenize sentences
    sents = nltk.sent_tokenize(article)
    # remove title (not needed for summarization task)
    sents = sents[1:]
    
    # get words per document
    # either use all words or only nouns
    if use_only_nouns: 
        #tagger = SennaChunkTagger('/usr/share/senna-v3.0/senna')
        words_per_doc = list()
        for sent in sents: 
            words = tokenizer.tokenize(sent.lower())
            
            # senna chunk tagging of nouns 
            # not sure if needed, just a test
            #tags = tagger.tag(words)
            #chunks = list(tagger.bio_to_chunks(tags, chunk_type='N'))

            # nltk pos tag 
            tagged_words = nltk.pos_tag(words)
            named_entities = nltk.ne_chunk(tagged_words)
            for word, tag in named_entities:
                if 'NN' in tag: 
                    words_per_doc.append(word)
    else: 
        words_per_sent = [set(tokenizer.tokenize(sent.lower())) for sent in sents]
        words_per_doc = flatten(words_per_sent)

    doc_tf_idf = defaultdict(str)
    for word in words_per_doc: 
        # inverse document frequency (idf)
        idf = math.log10(N/len(inverted_index[word]))
        # term frequency (tf) by document
        # get it from inverted index 
        doc_tf = inverted_index[word][article_id]
        doc_tf = 1 + math.log10(doc_tf)

        # tf-idf for the document 
        doc_tf_idf[word] = doc_tf * idf
    
    doc_tf_idf = dict(sorted(doc_tf_idf.items(), key=lambda item: item[1], reverse=True)[:max_num])
    for word in doc_tf_idf: 
        print(f"{word}: {doc_tf_idf[word]}")
    print(doc_tf_idf)

keyword_extraction(articles[0], 10, inverted_index, len(articles), 0, use_only_nouns=True)
        


NameError: name 'inverted_index' is not defined

# Evaluation

TODO:
* Implement evaluation
* Evaluation:
    * Statistics 
    * F-meassure
    * Recall-precision-curve
    * MAP
    * Efficiency

In [ ]:
'''
evaluation(Sset,Rset,args)
    @input the set of summaries Sset produced from selected documents Dset ⊆ D
    (e.g. a single document, a category of documents, the whole collection),
    the corresponding reference extracts Rset, and optional arguments (evalu-
    ation, preprocessing, model options)

    @behavior assesses the produced summaries against the reference ones using the tar-
    get evaluation criteria

    @output evaluation statistics, including F-measuring at predefined p-or-l summary
    limits, recall-and-precision curves, MAP, and efficiency
'''

'\nevaluation(Sset,Rset,args)\n    @input the set of summaries Sset produced from selected documents Dset ⊆ D\n    (e.g. a single document, a category of documents, the whole collection),\n    the corresponding reference extracts Rset, and optional arguments (evalu-\n    ation, preprocessing, model options)\n\n    @behavior assesses the produced summaries against the reference ones using the tar-\n    get evaluation criteria\n\n    @output evaluation statistics, including F-measuring at predefined p-or-l summary\n    limits, recall-and-precision curves, MAP, and efficiency\n'

In [ ]:
def evaluation(S: list, R: list, args=None) -> list:
    # do evaluation... 
    pass

B) **Summarization**

*B.1 Summarization solution: results for a given document*

In [ ]:
#code, statistics and/or charts here
article_id = 0
print(articles[article_id])
summarization(articles[article_id], num_sent=5, order="rel", inverted_index=inverted_index, N=len(articles), article_id=article_id) 

Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye following an injury, wh

{19: 0.2495193303958334,
 5: 0.21019493577531412,
 2: 0.20079836196045564,
 4: 0.1995382299926325,
 7: 0.19033491063190966}

*B.2 IR models (TF-IDF, BM25 and EBRT)*

In [ ]:
#code, statistics and/or charts here

*B.3 Reciprocal rank funsion*

In [ ]:
#code, statistics and/or charts here

*B.4 Maximal Marginal Relevance*

In [ ]:
#code, statistics and/or charts here

C) **Keyword extraction**

In [ ]:
#code, statistics and/or charts here
article_id = 0
print(articles[article_id])
keyword_extraction(articles[0], 10, inverted_index, len(articles), 0, use_only_nouns=True)

Bond game fails to shake or stir

For gaming fans, the word GoldenEye evokes excited memories not only of the James Bond revival flick of 1995, but also the classic shoot-em-up that accompanied it and left N64 owners glued to their consoles for many an hour.

Adopting that hallowed title somewhat backfires on this new game, for it fails to deliver on the promise of its name and struggles to generate the original's massive sense of fun. This however is not a sequel, nor does it relate to the GoldenEye film. You are the eponymous renegade spy, an agent who deserts to the Bond world's extensive ranks of criminal masterminds, after being deemed too brutal for MI6. Your new commander-in-chief is the portly Auric Goldfinger, last seen in 1964, but happily running around bent on world domination. With a determination to justify its name which is even less convincing than that of Tina Turner's similarly-titled theme song, the game literally gives the player a golden eye following an injury, wh

D) **Evaluation**

In [ ]:
#code, statistics and/or charts here

<H3>Part II: questions materials (optional)</H3>

**(1)** Corpus *D* and summaries *S* description.

In [ ]:
#code, statistics and/or charts here

**(2)** Summarization performance for the overall and category-conditional corpora.

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>END</H3>